In [84]:
import openai

# set api key
openai.api_key_path = "text/key.txt"
cur_model = "text-davinci-002"
done = False
finish = False

curcon_path = 'mem/cur_convo.txt'
conv_path = 'text/conversations.txt'
res_path = 'text/response.txt'
ctx_path = 'mem/cntx_prompt.txt'
nl = '\n'

In [85]:

def searchConv():
    return


In [86]:
def callGPT(model=cur_model,prompt="",temp=0,freq=0.5,pres=0.5,tok=120):
    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=temp,
        presence_penalty=pres,
        frequency_penalty=freq,
        max_tokens=tok,

    )

    return completion.choices[0].text


In [87]:
def optConvo(context):
    prompt  = f"'{context}'\n Summary and detail list from conversation between user and Dean: "
    summary = callGPT(prompt=prompt,temp=0,freq=0.2,pres=0.2,tok=120)

    return summary.strip('\n')


In [88]:

while not done:

    YOUR_PROMPT = input("Enter prompt: ")
    if YOUR_PROMPT == "exit session":
        done = True
        break
    cur_client = f"USER: {YOUR_PROMPT}{nl}"
    #read in context
    with open(curcon_path,"r") as cc:
        CONTEXT = cc.read()
        lastlines = cc.readlines()
        if len(lastlines) > 2:
            lastlines = lastlines[-2]
        cc.close()

    lastStrings = ' '.join([str(elem) for elem in lastlines])
    #####

    #optimize converation
    optCONTEXT = optConvo(CONTEXT)
    ####
    
    with open(ctx_path, "r") as ctx:
        ctxPrompt = ctx.read()
        ctx.close()

    ctxPrompt = ctxPrompt.format(optCONTEXT,lastStrings,YOUR_PROMPT)

    #prompt
    completion = callGPT(prompt=ctxPrompt,temp=1,freq=2,pres=2,tok=120)

    completion = completion.strip('\n\"')
    cur_dean = f"DEAN: {completion}{nl}"

    #update convo
    with open(curcon_path,"a+") as cc:
        cc.write(cur_client)
        cc.close()

    
    with open(curcon_path,"a+") as cc:
        cc.write(cur_dean)
        cc.close()
        
    #rate response
    finish = False
    while not finish:
        rate = input("(+/-): ")
        if '+' in rate:
            cur_dean = cur_dean[:5] + "(+)" + cur_dean[5:]
            finish=True
        elif '-' in rate:
            cur_dean = cur_dean[:5] + "(-)" + cur_dean[5:]
            finish=True
        

    with open(conv_path, "a+") as conv:
        pro_res = cur_client + cur_dean
        conv.write(pro_res)
        conv.close()

  
    


# erase convo file
open(curcon_path, "w").close()
####




In [89]:
# with open(curcon_path,"r") as cc:
#         CONTEXT = cc.read()
#         lastlines = cc.readlines()
#         if len(lastlines) > 2:
#             lastlines = lastlines[-2]
#         cc.close()

# lastStrings = ' '.join([str(elem) for elem in lastlines])
#     #####

#     #optimize converation
# optCONTEXT = optConvo(CONTEXT)
# print(optCONTEXT)

In [90]:
    # hintGen = "Paraphrase the therapy clients intents based on the following string: " + '"' + YOUR_PROMPT + '"'

    # HINT = openai.Completion.create(
    #     model=cur_model,
    #     prompt=hintGen,
    #     temperature=0.3,
    #     presence_penalty=0.5,
    #     frequency_penalty=0.5,
    #     max_tokens=100,

    # )
